In [1]:
# importing packages. See https://github.com/BasisResearch/collab-creatures for repo setup
import logging
import os
import time
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

from collab.utils import find_repo_root

root = find_repo_root()
from collab.foraging import random_hungry_followers as rhf
from collab.foraging.toolkit import (
    add_velocities_to_data_object,
    construct_visibility,
    filter_by_visibility,
    generate_grid,
    generate_velocity_scores,
)

logging.basicConfig(format="%(message)s", level=logging.INFO)

# users can ignore smoke_test -- it's for automatic testing on GitHub, to make sure the notebook runs on future updates to the repository
smoke_test = "CI" in os.environ
num_frames = 5 if smoke_test else 50
num_svi_iters = 10 if smoke_test else 1000
num_samples = 10 if smoke_test else 1000


notebook_starts = time.time()

# from .trace import rewards_trace
# from .utils import generate_grid

In [2]:
random_foragers_sim = rhf.RandomForagers(
    grid_size=40,
    probabilities=[1, 2, 3, 2, 1, 2, 3, 2, 1],
    num_foragers=3,
    num_frames=num_frames,
    num_rewards=15,
    grab_range=3,
)

random_foragers_sim()

sim = random_foragers_sim

grid = generate_grid(sim.grid_size)

grid = grid.sample(frac=1, random_state=42)

sim.grid = grid

vis = construct_visibility(
    sim.foragers,
    sim.grid_size,
    visibility_range=10,
    time_shift=0,
    grid=grid,
)
sim.visibility_range = 10
sim.visibility = vis["visibility"]
sim.visibilityDF = vis["visibilityDF"]

add_velocities_to_data_object(sim)

/home/rafal/s78projects/collaborative-intelligence/collab/foraging/random_hungry_followers/random_foragers.py:163: UserWarning: Warning: forager movements truncated to grid size. Try running again, or increase grid size.
  warnings.warn(


In [3]:
# for b in range(1, sim.num_foragers + 1):

time_shift = 0
finders_tolerance = 2
info_time_decay = 3

filter_by_visibility(
    sim,
    subject=1,
    time_shift=0,
    visibility_restriction="visible",
    info_time_decay=1,
    finders_tolerance=1,
    filter_by_on_reward=False,
).head()

,x,y,time,forager,type,velocity_x,velocity_y,distance,out_of_range
0,16.0,23.0,1,2,random,0.0,0.0,7.810250,False
0,22.0,22.0,1,3,random,0.0,0.0,4.000000,False
1,12.0,24.0,2,2,random,-4.0,1.0,9.486833,False
1,24.0,20.0,2,3,random,2.0,-2.0,3.162278,False
2,9.0,27.0,3,2,random,-3.0,3.0,8.944272,False


In [5]:
vs = generate_velocity_scores(sim)
sim.velocity_scores = vs["velocity_scores"]
sim.velocity_scoresDF = vs["velocity_scoresDF"]

forager 1
subject_at_t       x     y  time  forager    type  velocity_x  velocity_y
0  22.0  18.0     1        1  random         0.0         0.0
subject_x_at_t 22.0
subject_y_at_t 18.0
